In [177]:
import pandas as pd
from datetime import timedelta
from IPython.display import HTML, display

In [11]:
date = pd.datetime.today() - timedelta(days=30)
org = 'jupyterhub'
repo = 'zero-to-jupyterhub-k8s'

In [204]:
# Definition of a stale issue/PR (how many days since last updated)
date = pd.to_datetime(date)
stale_days = 30
date_stale = date - timedelta(days=stale_days)

# Opened and Closed Issues and PRs

This report shows opened and closed issues / PRs over the last 30 days, with a
particular focus on items that haven't had a lot of discussion, or that have
been opened by people who aren't team-members. The goal of this page is to make
it easier to spot places where some input would be appreciated!

# Imports

In [205]:
import pandas as pd
from subprocess import run, PIPE
from watchtower import issues_
from glob import glob
import os.path as op
import shutil as sh
import sys
sys.path.append('../../scripts/')

## styling functions

In [186]:
def highlight_rows(val):
    """Highlights cells pertaining to non-members and w/ no comments"""
    disp = ''
    if val == 0 or val == "NONE":
        disp += 'background-color: #f7b7b7'
    elif val == "FIRST_TIME_CONTRIBUTOR":
        disp += 'background-color: red'
    return disp

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def print_html(msg):
    display(HTML('<h2 style="padding-left: 1em;">{}</h2>'.format(msg)))
    
pd.options.display.max_colwidth = 200

## Load data

In [146]:
# Issues
issues = issues_.load_issues(org, repo)
issues['user'] = issues['user'].map(lambda a: a['login'])

# New activity

## Opened issues

without responses in red

In [149]:
opened = issues.query('created_at > @date')
opened_issues = opened[opened['pull_request'].isna()]
opened_issues['closed'] = ~pd.isnull(opened_issues['closed_at'])

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [180]:
print_html('Opened issues: {}'.format(len(opened_issues)))

In [170]:
opened_issues[['user', 'author_association', 'comments', 'closed', 'html_url']]\
    .sort_values(['closed', 'comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,closed,html_url
manycoding,CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1150
AXington,NONE,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1130
vilhelmen,CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1124
akhmerov,NONE,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1145
KVSSetty,NONE,3,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1147
jmatuskey,NONE,4,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1141
hnykda,NONE,5,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1125
aicampbell,NONE,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1131
LotteWeerts,NONE,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1148
hhuuggoo,NONE,1,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1139


## Closed issues


In [171]:
closed = issues.query('closed_at > @date')
closed_issues = closed[closed['pull_request'].isna()]

In [172]:
closed_issues[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
aicampbell,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1131
LotteWeerts,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1148
hhuuggoo,NONE,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1139
tregin,NONE,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1116
consideRatio,MEMBER,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1108
manics,CONTRIBUTOR,2,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1126
consideRatio,MEMBER,2,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1123
stelford,NONE,3,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1134
danielchalef,NONE,3,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1111
arokem,CONTRIBUTOR,4,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1118


In [181]:
print_html('Closed issues: {}'.format(len(closed_issues)))

## Opened PRs

without responses in red

In [161]:
opened_prs = opened[~opened['pull_request'].isna()]
opened_prs['closed'] = ~pd.isnull(opened_prs['closed_at'])

closed_prs = closed[~closed['pull_request'].isna()]

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [184]:
print_html('Opened PRs: {}'.format(len(opened_prs)))

In [187]:
opened_prs[['user', 'author_association', 'comments', 'closed', 'html_url']]\
    .sort_values(['closed', 'comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,closed,html_url
manycoding,CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1153
manycoding,CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1152
tmshn,CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1144
koen92,FIRST_TIME_CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1142
dhirschfeld,FIRST_TIME_CONTRIBUTOR,0,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1149
hnykda,FIRST_TIME_CONTRIBUTOR,3,False,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1136
manycoding,CONTRIBUTOR,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1138
choldgraf,MEMBER,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1133
consideRatio,MEMBER,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1128
manics,CONTRIBUTOR,0,True,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1127


In [185]:
print_html('Closed PRs: {}'.format(len(closed_prs)))

In [165]:
closed_prs[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
minrk,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1143
manycoding,CONTRIBUTOR,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1138
choldgraf,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1133
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1128
manics,CONTRIBUTOR,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1127
hnykda,FIRST_TIME_CONTRIBUTOR,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1112
manics,CONTRIBUTOR,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1113
ryanlovett,MEMBER,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1115
lesteve,CONTRIBUTOR,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1117
arokem,CONTRIBUTOR,1,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1119


# State of repo

## Stale issues

In [197]:
stale_issues = issues.query('updated_at < @date_stale')
stale_issues = stale_issues[stale_issues['closed_at'].isnull()]

In [201]:
stale_issues[['user', 'author_association', 'comments', 'html_url']]\
    .sort_values(['comments']).style.applymap(highlight_rows).format({'html_url': make_clickable}).hide_index()

user,author_association,comments,html_url
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1067
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1018
kevbutler,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1016
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/845
IamViditAgarwal,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1006
ablekh,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/998
frouzbeh,NONE,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/997
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/993
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/991
consideRatio,MEMBER,0,https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/861
